In [1]:
# -*- coding: utf-8 -*- 
import os  
import tarfile #解压用
from six.moves import urllib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import hashlib
import sklearn
from sklearn.model_selection import train_test_split
from pandas.tools.plotting import scatter_matrix
import cv2
import gdal
from sklearn.impute import SimpleImputer
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pickle

In [2]:
strat_train_set = pd.read_csv(".\datasets\strat_train_set.csv",index_col = 0)
housing= strat_train_set.drop("median_house_value",axis = 1) #axis = 1(列)
housing_num = housing.drop("ocean_proximity", axis = 1)
housing_labels = strat_train_set["median_house_value"].copy()
housing_labels.shape



(16512,)

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
# 为了能放入pipeline

# 列索引
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):  # 这个变量要传
        self.add_bedrooms_per_room = add_bedrooms_per_room # 控制是否加入bedrooms_per_room这个变量
    def fit(self, X, y=None):
        return self  # fit啥也不干
    # 通过数据计算的值添加进去
    def transform(self, X):
        # 先加两列
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        # 判断一下是否加第三列
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
            # 是的话返回三列
        else:
            return np.c_[X, rooms_per_household, population_per_household]
            # 否则返回两列

In [4]:
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

In [5]:
#第一种补充编码
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

#中位数填充
#加入特征
#将数据标准化
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])
housing_num_tr = num_pipeline.fit_transform(housing_num)
housing_num_tr[9]

array([ 0.64733449, -0.75752232,  0.02758786, -0.98565964, -0.74240006,
        1.13853483, -0.74541742, -1.77211596, -1.17438743,  0.80278786,
        3.53453121])

In [7]:
#第二种 进行onehot编码
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared =full_pipeline.fit_transform(housing)
housing_prepared.shape
housing_labels.shape

(16512,)

In [8]:
#将做好的数据集 存储为pickle
list_file = open('./datasets/housing_prepared.pickle','wb')
pickle.dump(housing_prepared,list_file)
list_file.close()



In [9]:
list_file = open('./datasets/housing_labels.pickle','wb')
pickle.dump(housing_labels,list_file)
list_file.close()